<a href="https://colab.research.google.com/github/Suchitra-V31/Natural_Language_processing/blob/main/Fake_News_Classifier_using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fake News Classifier using Bidirectional LSTM**



In this notebook we are going to use Fake News Classifier dataset and try to predict whether the given news is fake or not. Fro this we are going to use BidirectionalLSTM.

In [ ]:
import pandas as pd
import numpy as np

**LOAD DATA**

In [ ]:
data=pd.read_csv('/content/train.csv')

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.shape

(20800, 5)

In [ ]:
data.isnull().sum() # check whether any nan values are present

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
data=data.dropna() #drop all the nan values

In [ ]:
X=data.drop('label',axis=1)
y=data['label']

In [ ]:
X.shape,y.shape

((18285, 4), (18285,))

In [ ]:
# import the libraries
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.layers import Dense,Embedding,LSTM,Bidirectional

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Set Hyperparameters**

In [ ]:
#train
epoch=10
batch_size=64
#embedding
n_dim=40
n_unique_words=5000
max_review_len=20
pad_type='pre'
#lstm
n_lstm=100
n_drop=0.3



**Data Preprocessing**


In [ ]:
messages=X.copy()
messages.reset_index(inplace=True)

In [ ]:
from nltk.stem.porter import PorterStemmer


In [ ]:
stemmer=PorterStemmer()

In [ ]:
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[a-zA-Z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[stemmer.stem(word) for word in review if word not in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
one_hot_repr=[one_hot(words,n_unique_words) for words in corpus] #to convert text to numbers

In [ ]:
embed_doc=pad_sequences(one_hot_repr,padding=pad_type,maxlen=max_review_len)

**Design Neural Network Architecture**

In [ ]:
model=Sequential()
model.add(Embedding(n_unique_words,n_dim,input_length=max_review_len))
model.add(Bidirectional(LSTM(n_lstm,dropout=n_drop)))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


**Compile Model**

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
X_final=np.array(embed_doc)
y_final=np.array(y)

**Split the data into train/test**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

**TRAIN!!!!**

In [ ]:
model.fit(X_train,y_train,batch_size=batch_size,epochs=epoch,validation_data=(X_test,y_test))

Epoch 1/10
192/192 [==============================] - 5s 12ms/step - loss: 0.5630 - accuracy: 0.6765 - val_loss: 0.5763 - val_accuracy: 0.6696
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.5607 - accuracy: 0.6774 - val_loss: 0.5751 - val_accuracy: 0.6706
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.5591 - accuracy: 0.6762 - val_loss: 0.5778 - val_accuracy: 0.6701
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.5575 - accuracy: 0.6771 - val_loss: 0.5760 - val_accuracy: 0.6699
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.5563 - accuracy: 0.6778 - val_loss: 0.5773 - val_accuracy: 0.6691
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.5559 - accuracy: 0.6773 - val_loss: 0.5761 - val_accuracy: 0.6673
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.5559 - accuracy: 0.6777 - val_loss: 0.5768 - val_accuracy: 0.6681
Epoch

We could see that our model has predicted with 67% accuracy.